In [1]:
# from data import *
from model import PTModel
import torch.nn.functional as F
import torch
import torch.nn as nn 
import torchvision.transforms as T
import glob
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
from io import BytesIO
import random

In [4]:

def _is_pil_image(img):
    return isinstance(img, Image.Image)

def _is_numpy_image(img):
    return isinstance(img, np.ndarray) and (img.ndim in {2, 3})


class RandomHorizontalFlip(object):
    def __call__(self, sample):
        image, depth = sample['image'], sample['depth']

        if not _is_pil_image(image):
            raise TypeError(
                'img should be PIL Image. Got {}'.format(type(image)))
        if not _is_pil_image(depth):
            raise TypeError(
                'img should be PIL Image. Got {}'.format(type(depth)))

        if random.random() < 0.5:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)
            depth = depth.transpose(Image.FLIP_LEFT_RIGHT)

        return {'image': image, 'depth': depth}

class RandomChannelSwap(object):
    def __init__(self, probability):
        from itertools import permutations
        self.probability = probability
        self.indices = list(permutations(range(3), 3))

    def __call__(self, sample):
        image, depth = sample['image'], sample['depth']
        if not _is_pil_image(image): raise TypeError('img should be PIL Image. Got {}'.format(type(image)))
        if not _is_pil_image(depth): raise TypeError('img should be PIL Image. Got {}'.format(type(depth)))
        if random.random() < self.probability:
            image = np.asarray(image)
            image = Image.fromarray(image[...,list(self.indices[random.randint(0, len(self.indices) - 1)])])
        return {'image': image, 'depth': depth}

def loadZipToMem(zip_file):
    # Load zip file into memory
    print('Loading dataset zip file...', end='')
    from zipfile import ZipFile
    input_zip = ZipFile(zip_file)
    data = {name: input_zip.read(name) for name in input_zip.namelist()}
    nyu2_train = list((row.split(',') for row in (data['data/nyu2_train.csv']).decode("utf-8").split('\n') if len(row) > 0))

    from sklearn.utils import shuffle
    nyu2_train = shuffle(nyu2_train, random_state=0)

    #if True: nyu2_train = nyu2_train[:40]

    print('Loaded ({0}).'.format(len(nyu2_train)))
    return data, nyu2_train

class depthDatasetMemory(Dataset):
    def __init__(self, data, nyu2_train, transform=None):
        self.data, self.nyu_dataset = data, nyu2_train
        self.transform = transform

    def __getitem__(self, idx):
        sample = self.nyu_dataset[idx]
        image = Image.open( BytesIO(self.data[sample[0]]) )
        depth = Image.open( BytesIO(self.data[sample[1]]) )
        sample = {'image': image, 'depth': depth}
        if self.transform: sample = self.transform(sample)
        return sample

    def __len__(self):
        return len(self.nyu_dataset)

class ToTensor(object):
    def __init__(self,is_test=False):
        self.is_test = is_test

    def __call__(self, sample):
        image, depth = sample['image'], sample['depth']
        
        # image = self.to_tensor(image)

        depth = depth.resize((320, 240))
        print(np.array(depth))
        if self.is_test:
            depth = self.to_tensor(depth).float() / 1000
        else:            
            depth = self.to_tensor(depth).float() * 10000
        print(depth)
        # put in expected range
        depth = torch.clamp(depth, 10, 1000)
        # print(depth)
        return {'image': image, 'depth': depth}

    def to_tensor(self, pic):
        if not(_is_pil_image(pic) or _is_numpy_image(pic)):
            raise TypeError(
                'pic should be PIL Image or ndarray. Got {}'.format(type(pic)))

        if isinstance(pic, np.ndarray):
            print('Im here at isinstance')
            img = torch.from_numpy(pic.transpose((2, 0, 1)))

            return img.float().div(255)

        # handle PIL Image
        if pic.mode == 'I':
            print('pic mode I')
            img = torch.from_numpy(np.array(pic, np.int32, copy=False))
        elif pic.mode == 'I;16':
            print('pic mode I;16')
            img = torch.from_numpy(np.array(pic, np.int16, copy=False))
        else:
            print('pic mode else')
            img = torch.ByteTensor(
                torch.ByteStorage.from_buffer(pic.tobytes()))
        # PIL image mode: 1, L, P, I, F, RGB, YCbCr, RGBA, CMYK
        if pic.mode == 'YCbCr':
            nchannel = 3
        elif pic.mode == 'I;16':
            nchannel = 1
        else:
            nchannel = len(pic.mode)
        img = img.view(pic.size[1], pic.size[0], nchannel)

        img = img.transpose(0, 1).transpose(0, 2).contiguous()
        if isinstance(img, torch.ByteTensor):
            print('divided')
            return img.float().div(255)
        else:
            return img

def getNoTransform(is_test=False):
    return transforms.Compose([
        ToTensor(is_test=is_test)
    ])

def getDefaultTrainTransform():
    return transforms.Compose([
        RandomHorizontalFlip(),
        RandomChannelSwap(0.5),
        ToTensor()
    ])

class depthDataset(Dataset):
    def __init__(self, transform=None, is_test=False):
        self.data = []
        # self.data = dataset
        if not is_test: 
            with open('train.txt', 'r') as f:
                self.data = f.readlines()
                self.data = [line.rstrip() for line in self.data]
        else:  
            with open('test.txt', 'r') as f:
                self.data = f.readlines()
                self.data = [line.rstrip() for line in self.data]  
        self.transform = transform

    def __getitem__(self, idx):
        sample = self.data[idx]
        # print(self.data[idx], type(self.data[idx]), len(self.data[idx]))
        image = Image.open( self.data[idx].split('\t')[0] )
        depth = Image.open( self.data[idx].split('\t')[1] )
        # print(depth)
        sample = {'image': image, 'depth': depth}
        if self.transform: sample = self.transform(sample)
        return sample

    def __len__(self):
        return len(self.data)

def getTrainingTestingData(batch_size):
    data, nyu2_train = loadZipToMem('nyu_data.zip')

    transformed_training = depthDatasetMemory(data, nyu2_train, transform=getDefaultTrainTransform())
    transformed_testing = depthDatasetMemory(data, nyu2_train, transform=getNoTransform())

    return DataLoader(transformed_training, batch_size, shuffle=True), DataLoader(transformed_testing, batch_size, shuffle=False)


In [3]:
data, nyu = loadZipToMem('nyu_data.zip')



Loading dataset zip file...Loaded (50688).


In [3]:
transformed_training = depthDatasetMemory(data, nyu, transform=getDefaultTrainTransform())

NameError: name 'data' is not defined

In [14]:
# sample = transformed_training.nyu_dataset[0]
# depth = Image.open( BytesIO(transformed_training.data[sample[1]]) )
# np.array(depth)
x = transformed_training[0]

pic mode else
divided
[[24 24 24 ... 30 30 30]
 [24 24 24 ... 30 30 30]
 [24 24 24 ... 30 30 30]
 ...
 [27 27 27 ... 21 21 21]
 [27 27 27 ... 21 21 21]
 [27 27 27 ... 21 21 21]]
pic mode else
divided
tensor([[[ 94.1176,  94.1176,  94.1176,  ..., 117.6471, 117.6471, 117.6471],
         [ 94.1176,  94.1176,  94.1176,  ..., 117.6471, 117.6471, 117.6471],
         [ 94.1176,  94.1176,  94.1176,  ..., 117.6471, 117.6471, 117.6471],
         ...,
         [105.8824, 105.8824, 105.8824,  ...,  82.3529,  82.3529,  82.3529],
         [105.8824, 105.8824, 105.8824,  ...,  82.3529,  82.3529,  82.3529],
         [105.8824, 105.8824, 105.8824,  ...,  82.3529,  82.3529,  82.3529]]])


In [7]:
print(nyu[0])

['data/nyu2_train/bathroom_0053_out/66.jpg', 'data/nyu2_train/bathroom_0053_out/66.png']


In [ ]:
rgb = '/media/krishna/Data/generated_2.1.0/train/*/*/raw_images/*'
depth_images = '/media/krishna/Data/generated_2.1.0/train/*/*/depth_images/*'

files = {'depth': [], 'rgb': []}
for name in glob.glob(pathname=depth_images):
    files['depth'].append(name)

for name in glob.glob(pathname=rgb):
    files['rgb'].append(name)

print(len(files['depth']), len(files['rgb']))

In [ ]:
import random
# idx = random.randint(0,333623)
dataset = []
for i in range(60000):
    idx = random.randint(0,333623)
    rgb, depth = files['rgb'][idx], files['depth'][idx]
    dataset.append([rgb, depth])

with open('train.txt', 'w') as f:
    for i in range(50000):
        f.write("%s\t%s\n" % (dataset[i][0], dataset[i][1]))


with open('test.txt', 'w') as f:
    for i in range(50000, 60000):
        f.write("%s\t%s\n" % (dataset[i][0], dataset[i][1]))
# print(rgb, depth)

In [ ]:
path = 'Unet_depth20.pth'
model = PTModel()
model.load_state_dict(torch.load(path))
print('Model successfully loaded.')

transformed_testing = depthDataset(transform=getNoTransform(), is_test=True)

y = model(transformed_testing[0]['image'].unsqueeze(0))
im = np.array(y[0][0].detach().cpu().numpy())
# im = 1./im
im = np.divide(5000,im)
print(im)

In [5]:
transformed_training = depthDataset(transform=getDefaultTrainTransform(), is_test=False)

FileNotFoundError: [Errno 2] No such file or directory: 'train.txt'

In [4]:
transformed_training[0]['depth'].shape

torch.Size([1, 300, 300])

In [9]:
transform = T.Compose([T.PILToTensor()])
transform(transformed_training[0]['depth'])

<PIL.PngImagePlugin.PngImageFile image mode=L size=300x300 at 0x7F10563F7D90>


tensor([[[28, 28, 28,  ..., 29, 29, 29],
         [28, 28, 28,  ..., 29, 29, 29],
         [28, 28, 28,  ..., 29, 29, 29],
         ...,
         [36, 36, 36,  ..., 36, 36, 36],
         [36, 36, 36,  ..., 36, 36, 36],
         [36, 36, 36,  ..., 36, 36, 36]]], dtype=torch.uint8)